In [20]:
import cv2
import mediapipe as mp
import pyttsx3
import pygame
from scipy.spatial import distance

# Initialize pyttsx3 for alert audio message
engine = pyttsx3.init()

# Initialize pygame mixer for sound
pygame.mixer.init()
alert_sound_path = r"C:\Users\asmit\Documents\python basics\.ipynb_checkpoints\Drowsiness Detection\beep-beep-beep-beep-80262.mp3"
pygame.mixer.music.load(alert_sound_path)

# Mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to calculate the aspect ratio for the eye
def detect_eye(landmarks, eye_points):
    poi_A = distance.euclidean(landmarks[eye_points[1]], landmarks[eye_points[5]])
    poi_B = distance.euclidean(landmarks[eye_points[2]], landmarks[eye_points[4]])
    poi_C = distance.euclidean(landmarks[eye_points[0]], landmarks[eye_points[3]])
    aspect_ratio_eye = (poi_A + poi_B) / (2 * poi_C)
    return aspect_ratio_eye

# Eye landmarks based on Mediapipe
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# Drowsiness detection thresholds
EAR_THRESHOLD = 0.25  # Adjust based on testing
FRAMES_THRESHOLD = 20  # Number of consecutive frames to confirm drowsiness

# Counter for frames with eyes closed
closed_eye_frames = 0

# Capture from webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame for a mirrored view and convert to RGB
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract landmark coordinates
            landmarks = [(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0]))
                         for lm in face_landmarks.landmark]

            # Get the bounding box of the face
            x_min = min([lm[0] for lm in landmarks])
            x_max = max([lm[0] for lm in landmarks])
            y_min = min([lm[1] for lm in landmarks])
            y_max = max([lm[1] for lm in landmarks])

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Calculate eye aspect ratio
            left_eye_ratio = detect_eye(landmarks, LEFT_EYE)
            right_eye_ratio = detect_eye(landmarks, RIGHT_EYE)
            eye_ratio = (left_eye_ratio + right_eye_ratio) / 2

            # Check if the eye ratio is below the threshold
            if eye_ratio < EAR_THRESHOLD:
                closed_eye_frames += 1
            else:
                closed_eye_frames = 0

            # Set drowsiness status
            if closed_eye_frames >= FRAMES_THRESHOLD:
                status = "Drowsy"
                cv2.putText(frame, "DROWSINESS DETECTED", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(frame, "Alert! Take a Small Break!", (50, 450),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Play alert sound
                if not pygame.mixer.music.get_busy():  # Check if sound is already playing
                    pygame.mixer.music.play()

                # Text-to-speech alert
                engine.say("Alert! Wake up!")
                engine.runAndWait()
            else:
                status = "Awake"

            # Display the status in the top-left corner
            cv2.putText(frame, f"Status: {status}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Drowsiness Detector", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        pygame.mixer.music.stop()  # Stop the sound immediately
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()
